In [ ]:
import csv
import numpy as np
import pandas as pd


In [ ]:
'''
The below chunk of code, drops null values from all datasets, turns the year_of_publications col to int64,
drops all images and image col from df's, converts all col to string (excluding year_of_pub), drops null values from ratings col 
'''

# Replace 'your_file.csv' with the path to your CSV file
file_paths = ['Users.csv', 'Books.csv', 'Ratings.csv']

# Load data and drop rows with null values
userData = pd.read_csv("data/"+file_paths[0], quoting=csv.QUOTE_MINIMAL, quotechar='"').dropna()


booksData = pd.read_csv("data/"+file_paths[1], quoting=csv.QUOTE_MINIMAL, quotechar='"', 
                        converters={'Year-Of-Publication': lambda x: pd.to_numeric(x, errors='coerce')}).dropna()

# Drop specified columns
columns_to_drop = ['Image-URL-S', 'Image-URL-M', 'Image-URL-L']
booksData.drop(columns=columns_to_drop, inplace=True)
booksData['Year-Of-Publication'] = booksData['Year-Of-Publication'].astype('Int64')

# Select columns to convert to strings (excluding 'Year-Of-Publication')
columns_to_convert = [col for col in booksData.columns if col != 'Year-Of-Publication']

ratingsData = pd.read_csv("data/"+file_paths[2], quoting=csv.QUOTE_MINIMAL, quotechar='"').dropna()


# Display data  of all columns in booksData
print("user data")
print(50 * "=")
print(userData)
print("book data")
print(50 * "=")
print(booksData)
print("rating data")
print(50 * "=")
print(ratingsData)

